In [ ]:
"""
NOTEBOOK 08: MÉTRICAS CLÍNICAS Y COMPARACIÓN
==============================================
Calcular indicadores clínicos de perfusión tisular a partir
de mediciones microvasculares y comparar grupos de pacientes.

Métricas clínicas:
- Perfusion Index (PI) 
- Microvascular Flow Index (MFI)
- Proportion of Perfused Vessels (PPV)
- Heterogeneity Index (HI)

Análisis comparativo:
- Cambios día-a-día por paciente
- Diferencias grupo responders vs. no-responders
- Análisis de supervivencia y outcomes

Entrada: Características de todos los notebooks previos
Salida: Reporte de métricas clínicas, análisis de grupos

Autor: Tesis Doctoral - Análisis Automatizado de Dinámicas Microvasculares
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from sklearn.metrics import roc_auc_score, confusion_matrix

# Configuración de visualización
sns.set_style("whitegrid")
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['legend.fontsize'] = 10

In [ ]:
# Save clinical metrics
output_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/clinical_outcomes.csv')
df_clinical.to_csv(output_path, index=False)

print(f"\nClinical outcomes saved to: {output_path}")
print("\nReady for final summary in Notebook 09")

In [ ]:
# Statistical comparisons
print("\n\nStatistical Analysis:")
print("=" * 80)

# Compare groups
class_groups = df_clinical.groupby('clinical_classification')

for metric in ['mean_svd', 'mean_tvd', 'pct_continuous_flow']:
    print(f"\n{metric}:")
    for classification, group in class_groups:
        values = group[metric].values
        print(f"  {classification}: {np.mean(values):.2f} ± {np.std(values):.2f}")

# Visualize clinical outcomes
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Clinical Metrics and Patient Classification', fontsize=14, fontweight='bold')

# SVD by classification
ax = axes[0, 0]
colors_class = {'Good': 'green', 'Moderate': 'orange', 'Poor': 'red'}
for classification in df_clinical['clinical_classification'].unique():
    data = df_clinical[df_clinical['clinical_classification'] == classification]
    ax.scatter(range(len(data)), data['mean_svd'], label=classification, 
              s=200, alpha=0.7, color=colors_class[classification])
ax.set_ylabel('Mean SVD')
ax.set_title('Small Vessel Density by Prognosis')
ax.legend()
ax.grid(True, alpha=0.3)

# Continuous flow percentage
ax = axes[0, 1]
for classification in df_clinical['clinical_classification'].unique():
    data = df_clinical[df_clinical['clinical_classification'] == classification]
    ax.scatter(range(len(data)), data['pct_continuous_flow'], label=classification,
              s=200, alpha=0.7, color=colors_class[classification])
ax.set_ylabel('% Continuous Flow')
ax.set_title('Vessel Perfusion Quality')
ax.legend()
ax.grid(True, alpha=0.3)

# Classification summary
ax = axes[1, 0]
class_counts = df_clinical['clinical_classification'].value_counts()
ax.bar(class_counts.index, class_counts.values, color=[colors_class[c] for c in class_counts.index], alpha=0.7)
ax.set_ylabel('Number of Patients')
ax.set_title('Patient Classification Distribution')
ax.grid(True, alpha=0.3, axis='y')

# Scatter: SVD vs continuous flow
ax = axes[1, 1]
for classification in df_clinical['clinical_classification'].unique():
    data = df_clinical[df_clinical['clinical_classification'] == classification]
    ax.scatter(data['mean_svd'], data['pct_continuous_flow'], label=classification,
              s=200, alpha=0.7, color=colors_class[classification])
ax.set_xlabel('Mean SVD')
ax.set_ylabel('% Continuous Flow')
ax.set_title('SVD vs Perfusion Quality')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/clinical_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

# Load all data
annotations_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/annotations_processed.csv')
df_annotations = pd.read_csv(annotations_path)

spacetime_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/spatiotemporal_features.csv')
df_spacetime = pd.read_csv(spacetime_path)

dynamics_path = Path('/Users/luisestebanbaldasseroni/LuisEsteban/tesis/microcirculation-analysis/src/data/dynamics_models.csv')
if dynamics_path.exists():
    df_dynamics = pd.read_csv(dynamics_path, index_col=0)
else:
    df_dynamics = None

# Clinical outcome classification
clinical_outcomes = []

for patient in df_annotations['patient'].unique():
    patient_data = df_annotations[df_annotations['patient'] == patient]
    
    # Compute clinical metrics
    mean_svd = patient_data['svd'].mean()
    mean_tvd = patient_data['tvd'].mean()
    mean_vessels = patient_data['total_vessels'].mean()
    
    # Percentage of continuous flow (cat 3)
    total_count = patient_data[['count_cat0', 'count_cat1', 'count_cat2', 'count_cat3']].sum(axis=1).sum()
    continuous_flow = patient_data['count_cat3'].sum()
    pct_continuous = 100 * continuous_flow / total_count if total_count > 0 else 0
    
    # Classify outcome
    if mean_svd > 15 and pct_continuous > 70:
        classification = 'Good'
    elif mean_svd < 10 or pct_continuous < 30:
        classification = 'Poor'
    else:
        classification = 'Moderate'
    
    clinical_outcomes.append({
        'patient': patient,
        'mean_svd': mean_svd,
        'mean_tvd': mean_tvd,
        'mean_vessels': mean_vessels,
        'pct_continuous_flow': pct_continuous,
        'num_records': len(patient_data),
        'clinical_classification': classification
    })

df_clinical = pd.DataFrame(clinical_outcomes)

print("Clinical Metrics Summary by Patient:")
print(df_clinical.to_string(index=False))

# Notebook 08: Clinical Metrics and Patient Comparison

Computes standardized clinical metrics and performs statistical comparisons between patient groups.

# 08 - Comparación con AVA y Métricas Clínicas

## Objetivo
Comparar outputs con AVA 3.2 o análisis manual:
- MFI, PPV, PVD
- Coherencia hemodinámica


In [ ]:
import seaborn as sns
import pandas as pd

df = pd.read_csv("results_comparison.csv")
sns.scatterplot(data=df, x="PVD_AVA", y="PVD_model")
plt.plot([0, df.max().max()], [0, df.max().max()], 'r--')
plt.title("Comparación de PVD")
